In [1]:
# Part 1 - Building the CNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
# Initialising the CNN
classifier = Sequential()

In [3]:
# Step 1 - Convolution
# in this step feature detector is passed all over the image to create a feature map this is done multiple times
# to create multiple feature maps
# in ann dense was use but that is used in case of fully connected layer here that is not the case
# 32 is number of filters i.e. number of feature maps
# 3,3 is number of rows and columns of feature detector 
# niput shape is specified but it is done during datset processing
classifier.add(Convolution2D(32, 3, 3, input_shape = (64, 64, 3), activation = 'relu'))

/home/bhawesh/.local/lib/python3.5/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(64, 64, 3..., activation="relu")`
  


In [4]:
# Step 2 - Pooling
# in max pooling we basically reduce the size of feature map, we pass the 2*2 matrix over feature map
# and select the max value from the matrix

classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [5]:
# Adding a second convolutional layer
# this is added to increase the accuracy of the model we can either do this or add another fully connected layer
# in a new convolution layer we don't need to specify the input shape as the input is being passed
# through max pooling step
classifier.add(Convolution2D(32, 3, 3, activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

/home/bhawesh/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  


In [6]:
# Step 3 - Flattening
# during flattening we don't lose the features as during convolution we have extracted most of the features
# so during flattening i.e. converting to 1-d array
# we could've have directly flattened the image but this will not tell us the feature of the image
# like it does after max pooling on other steps. Whereas orignal image just tells us the feature of the single 
# pixel which not very relevant compared to spatial feature extracted after convolution steps

classifier.add(Flatten())

In [7]:
# Step 4 - Full connection
# now we need a fully connected layer to get the output in case of dog and cat classification
# output dim is selected as 128 as it is good practice to select a number above 100
# next activation is sigmoid since binary classification is present in case of multiple category softmax is used

classifier.add(Dense(output_dim = 128, activation = 'relu'))
classifier.add(Dense(output_dim = 1, activation = 'sigmoid'))

/home/bhawesh/.local/lib/python3.5/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=128, activation="relu")`
  
/home/bhawesh/.local/lib/python3.5/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1, activation="sigmoid")`
  import sys


In [8]:
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [10]:
# Part 2 - Fitting the CNN to the images
'''
image augmentation is applied here to avoid overfitting as during training this will augment data and apply
some steps like rotating shearing etc
target size is size mentioned in CNN
class mode binay as only two classes present
samples_per_epoch is number of images in training
'''
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')



Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [14]:
# classifier.fit_generator(training_set,
#                          samples_per_epoch = 8000,
#                          nb_epoch = 25,
#                          validation_data = test_set,
#                          nb_val_samples = 2000)

classifier.fit_generator(
        training_set,
        steps_per_epoch=8000/32,
        epochs=25,
        validation_data=test_set,
        validation_steps=2000/32)

Epoch 1/25
250/250 [==============================] - 89s 357ms/step - loss: 0.5381 - acc: 0.7245 - val_loss: 0.5079 - val_acc: 0.7540
Epoch 2/25
250/250 [==============================] - 87s 347ms/step - loss: 0.5137 - acc: 0.7483 - val_loss: 0.5973 - val_acc: 0.6935
Epoch 3/25
250/250 [==============================] - 86s 344ms/step - loss: 0.4833 - acc: 0.7651 - val_loss: 0.4923 - val_acc: 0.7645
Epoch 4/25
250/250 [==============================] - 83s 333ms/step - loss: 0.4722 - acc: 0.7735 - val_loss: 0.4740 - val_acc: 0.7815
Epoch 5/25
250/250 [==============================] - 81s 324ms/step - loss: 0.4442 - acc: 0.7933 - val_loss: 0.4816 - val_acc: 0.7690
Epoch 6/25
250/250 [==============================] - 81s 324ms/step - loss: 0.4503 - acc: 0.7877 - val_loss: 0.4598 - val_acc: 0.7920
Epoch 7/25
250/250 [==============================] - 81s 324ms/step - loss: 0.4216 - acc: 0.8005 - val_loss: 0.4677 - val_acc: 0.7950
Epoch 8/25
250/250 [==============================] - 8

In [19]:
# for prediction

from keras.preprocessing import image as image_utils
import numpy as np
 
test_image = image_utils.load_img('dataset/c1.jpeg', target_size=(64, 64))
test_image = image_utils.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
 
result = classifier.predict_on_batch(test_image)
print(result) #dog=1 cat =0 

[[0.]]


In [21]:
# https://machinelearningmastery.com/save-load-keras-deep-learning-models/
from keras.models import model_from_json
# serialize model to JSON
model_json = classifier.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
classifier.save_weights("model.h5")
print("Saved model to disk")
 
# later...
 
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Saved model to disk
Loaded model from disk
